# MNIST classification using a multi-label representation

In [3]:
import os
import numpy as np       
import parse_log as pl
from caffe.eval_utils import Phase
import iow.file_system_utils as fs
from log_utils import is_caffe_info_log
    
def cleanup_caffe_logname(fname):
    
    fname = fname.replace('log', '')
    fname = fname.replace('caffe', '')
    fname = fname.replace('.', '')
    return fname

class LearningCurve(object):
    '''
    classdocs
    '''
    def list(self, key, phase=Phase.TEST):
                
        if phase.lower() == Phase.TEST.lower():
            
            if 'loss' in key.lower() and 'loss' in self.test_keys:
                key = 'loss'
            elif 'accuracy' in key.lower() and 'accuracy' in self.test_keys:
                key = 'accuracy'
                
            return np.array([[x[key]] for x in self.test_dict_list])
        else:
            return np.array([[x[key]] for x in self.train_dict_list])
        
    def name(self):
    
        name = os.path.basename(self.path_log)
        name, _ = os.path.splitext(name)
        return cleanup_caffe_logname(name)
    
    def parse(self):
        
        log_data = pl.parse_log(self.path_log)
        # allow for backwards compatibility
        if len(log_data) == 4:
            self.train_dict_list, self.train_keys, self.test_dict_list, self.test_keys = log_data
        else:
            self.train_dict_list, self.test_dict_list = log_data
            if len(self.train_dict_list) > 0:
                self.train_keys = self.train_dict_list[0].keys()
            else:
                self.train_keys = []
                
            if len(self.test_dict_list) > 0:
                self.test_keys = self.test_dict_list[0].keys()
            else:
                self.test_keys = []

        return self.train_keys, self.test_keys

    def __init__(self, path_log):
        '''
        Constructor
        '''
        self.path_log = path_log
        
def LearningCurveFromPath(p):
    
    if os.path.isfile(p):
        return LearningCurve(p)
    elif os.path.isdir(p):
        log_paths = fs.gen_paths(p, func_filter=is_caffe_info_log)
        if len(log_paths) > 0:
            return LearningCurve(log_paths[-1])
        else:
            return None
    else:
        raise IOError("%s: No such file or directory" % (p,))

/mnt/antares_raid/home/pierre/elnath/src/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/mnt/antares_raid/home/pierre/elnath/src/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/mnt/antares_raid/home/pierre/elnath/src/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


ImportError: No module named eval_utils

In [2]:
%pylab inline

#from eval.learning_curve import LearningCurve
#from eval.eval_utils import Phase
#import eval.log_utils as lu

from pylab import rcParams
rcParams['figure.figsize'] = 16, 6
rcParams.update({'font.size': 15})

def print_learning_curve(net_prefix, log_name, fig_name):

    e = LearningCurve(log_name)
    e.parse()

    for phase in [Phase.TRAIN, Phase.TEST]:
        num_iter = e.list('NumIters', phase)
        loss = e.list('loss', phase)
        plt.plot(num_iter, loss, label='on %s set' % (phase.lower(),))

        plt.xlabel('iteration')
        # format x-axis ticks
        ticks, _ = plt.xticks()
        plt.xticks(ticks, ["%dK" % int(t/1000) for t in ticks])
        plt.ylabel('loss')
        plt.title(net_prefix+' on train and test sets')
        plt.legend()
    plt.savefig(fig_name)

/mnt/antares_raid/home/pierre/.virtualenvs/pierrenv/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [ ]:
log_name = "ex"
print_learning_curve("net_hdf5", "logs/"+log_name, "imgs/essai.png")

In [ ]:
import caffe
solver = None
solver = caffe.SGDSolver("prototxt/net_hdf5_solver.prototxt")

print "Output dimensions:"
print [(k, v[0].data.shape) for k, v in solver.net.params.items()]

print "Weight sizes:"
print [(k, v[0].data.shape) for k, v in solver.net.params.items()]

solver.net.forward()
solver.net.forward()
solver.test_nets[0].forward()
print solver.test_nets[0].blobs['score'].data[0,:]

# we use a little trick to tile the first eight images
imshow(solver.net.blobs['data'].data[:8, 0].transpose(1, 0, 2).reshape(28, 8*28), cmap='gray'); axis('off')
print 'Train labels:', [np.argmax(label) for label in solver.net.blobs['label'].data[:8]]